# Digit classification using MNIST

This is a dataset of 60,000 28x28 grayscale images of the 10 digits, along with a test set of 10,000 images. More info can be found at the [MNIST homepage](http://yann.lecun.com/exdb/mnist/).

In [ ]:
# importing necessary modules
import tensorflow as tf
import seaborn as sns
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.cm as cm
%matplotlib inline

# dataset used in this example
from keras.datasets import mnist
from keras.utils import np_utils
# layers used in this example
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPool2D

## Load the dataset

* `x_train`: uint8 NumPy array of grayscale image data with shapes `(60000, 28, 28)`, containing the training data. Pixel values range from 0 to 255.

* `y_train`: uint8 NumPy array of digit labels (integers in range 0-9) with shape `(60000,)` for the training data.

* `x_test`: uint8 NumPy array of grayscale image data with shapes `(10000, 28, 28)`, containing the test data. Pixel values range from 0 to 255.

* `y_test`: uint8 NumPy array of digit labels (integers in range 0-9) with shape `(10000,)` for the test data.

In [ ]:
# loading the dataset and spliting into training and testing samples
(X_train, y_train), (X_test, y_test) = mnist.load_data()
# checking
assert X_train.shape == (60000, 28, 28)
assert X_test.shape == (10000, 28, 28)
assert y_train.shape == (60000,)
assert y_test.shape == (10000,)

## Exploratory data analysis

In [ ]:
# how many instances in each training class
sns.countplot(y_train)

In [ ]:
# checking for missing data
print(np.isnan(X_train).any(), np.isnan(X_test).any())

In [ ]:
# plot the first six training images
fig = plt.figure(figsize=(20,20))
for i in range(6):
    ax = fig.add_subplot(1, 6, i+1, xticks=[], yticks=[])
    ax.imshow(X_train[i]) # add cmap='gray' for gray scale images
    ax.set_title(str(y_train[i]))

## Data pre-processing

### Normalization

Models generally run better on normalized values. The best way to normalize the data depends on each individual dataset. For the MNIST dataset, we want each value to be between 0.0 and 1.0. As all values originally fall under the 0.0-255.0 range, divide by 255.0.

In [ ]:
# rescale [0,225] --> [0,1]
X_train = X_train.astype('float32')/255
X_test = X_test.astype('float32')/255

### Label Encoding

The labels for the training and the testing dataset are currently categorical (not continuous). To include a categorical dataset in our model, the labels should be converted to one-hot encodings.

For example, `2` becomes `[0,0,1,0,0,0,0,0,0,0]` and 7 becomes `[0,0,0,0,0,0,0,1,0,0]`.

In [ ]:
# print first ten (integer valued) training labels
print('Integer-valued labels: ')
print(y_train[:10])

In [ ]:
# one-hot encode the labels
y_train = np_utils.to_categorical(y_train, 10)
y_test = np_utils.to_categorical(y_test, 10)

# print first ten (one hot) training labels
print('One-hot labels: ')
print(y_train[:10])

In [ ]:
plt.imshow(X_train[100][:,:])
print("One-hot encoded class label: ", y_train[100])

### Backup data 

For this exercise, we will testing a single model (Dense layer) first (as baseline), and then experimenting with more specialised models. 

For the baseline model, we need to reshape the input data. This will be `X_train` and `X_test` samples. All the remaining models will use a copy of these data samples (`X_train2` and `X_test2`), with the original shape.

In [ ]:
# X_train2 and X_test2 will keep the original 3D tensor format (28,28,1) while X_train and X_test will be reshaped to a 1D tensor (784).
X_train2 = X_train
X_test2 = X_test

## General model definition and training parameters

In [ ]:
# Some useful parameters for model definition and training
batch_size = 64
num_classes = 10
epochs = 10

## Baseline model: 1D model

We can start with a single Dense layer model and check the performance.

In [ ]:
# we need to reshape the data, as the Dense layer expects a single 1D tensor as input
print("X_train original shape: ",  X_train.shape)
X_train = tf.reshape(X_train, [-1, 784])
print("X_train original shape: ", X_train.shape)

print("X_test original shape: ",  X_test.shape)
X_test = tf.reshape(X_test, [-1, 784])
print("X_test original shape: ", X_test.shape)

In [ ]:
# uncomment this line if you want to see the image
# remember to upload the image (from GitHub) into Google Colab
#from IPython.display import display, Image
#display(Image(filename='/content/w05_baselineCNNmodel.png'))

In [ ]:
# baseline model - single Dense layer
model1 = tf.keras.Sequential(
  [
      tf.keras.layers.Input(shape=(28*28)), # input should be only the dense vector (784 data points)
      tf.keras.layers.Dense(num_classes, activation='softmax')
  ])

model1.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])

# print model layers
model1.summary()

In [ ]:
%time
history = model1.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.3)

In [ ]:
# evaluate test accuracy
score = model1.evaluate(X_test, y_test, verbose=0)
accuracy = 100*score[1]

# print test accuracy
print('Test accuracy %.2f%%' % accuracy)

In [ ]:
fig, ax = plt.subplots(2,1)
ax[0].plot(history.history['loss'], color='b', label="Training Loss")
ax[0].plot(history.history['val_loss'], color='r', label="Validation Loss",axes =ax[0])
legend = ax[0].legend(loc='best', shadow=True)

ax[1].plot(history.history['accuracy'], color='b', label="Training Accuracy")
ax[1].plot(history.history['val_accuracy'], color='r',label="Validation Accuracy")
legend = ax[1].legend(loc='best', shadow=True)

### Comments on the baseline model

---

### Second model

We will add some convolutional and pooling layers to improve model's performance.

* `Conv2D` layers are convolutions. Each filter (32 in this example) transforms a part of the image (5x5 in this example). The transformation is applied on the whole image.

* `MaxPool2D` is a downsampling filter. It reduces a 2x2 matrix of the image to a single pixel with the maximum value of the 2x2 matrix. The filter aims to conserve the main features of the image while reducing the size.

* `relu` is the rectifier, and it is used to find nonlinearity in the data. It works by returning the input value if the input value >= 0. If the input is negative, it returns 0.

* `Flatten` converts the tensors into a 1D vector.

* The `Dense` layer is a fully-connected neural network (ANN). The last layer returns the probability that an image is in each class (one for each digit).

* As this model aims to categorize the images, we will use a `categorical_crossentropy` loss function.

In [ ]:
model2 = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), padding='same', activation='relu', input_shape=(28,28,1), name='1st_Conv2D'),
    tf.keras.layers.MaxPool2D(name='1st_MaxPool2D'),
    tf.keras.layers.Flatten(name='flatten'),
    tf.keras.layers.Dense(num_classes, activation='softmax', name='final_dense')
])

model2.summary()

In [ ]:
# compile the model
model2.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy']) # changed from rmsprop

### Training the model

In [ ]:
%time
history = model2.fit(X_train2, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.3)

### Model evaluation

In [ ]:
# evaluate test accuracy
score = model2.evaluate(X_test2, y_test, verbose=0)
accuracy = 100*score[1]

# print test accuracy
print('Test accuracy %.2f%%' % accuracy)

In [ ]:
history.history.keys()

In [ ]:
fig, ax = plt.subplots(2,1)
ax[0].plot(history.history['loss'], color='b', label="Training Loss")
ax[0].plot(history.history['val_loss'], color='r', label="Validation Loss",axes =ax[0])
legend = ax[0].legend(loc='best', shadow=True)

ax[1].plot(history.history['accuracy'], color='b', label="Training Accuracy")
ax[1].plot(history.history['val_accuracy'], color='r',label="Validation Accuracy")
legend = ax[1].legend(loc='best', shadow=True)

### Comments on the second model

---

### Third model

In this third model, we are adding a second `Conv2D` layer with 64 3x3 kernels and keep the remaining parameters.

In [ ]:
model3 = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), padding='same', activation='relu', input_shape=(28,28,1)),
    tf.keras.layers.Conv2D(64, (3,3), padding='same', activation='relu'),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(num_classes, activation='softmax')
])

model3.summary()

In [ ]:
model3.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy']) # changed from rmsprop

In [ ]:
%time
history = model3.fit(X_train2, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.3)

In [ ]:
# evaluate test accuracy
score = model3.evaluate(X_test2, y_test, verbose=0)
accuracy = 100*score[1]

# print test accuracy
print('Test accuracy %.4f%%' % accuracy)

In [ ]:
fig, ax = plt.subplots(2,1)
ax[0].plot(history.history['loss'], color='b', label="Training Loss")
ax[0].plot(history.history['val_loss'], color='r', label="Validation Loss",axes =ax[0])
legend = ax[0].legend(loc='best', shadow=True)

ax[1].plot(history.history['accuracy'], color='b', label="Training Accuracy")
ax[1].plot(history.history['val_accuracy'], color='r',label="Validation Accuracy")
legend = ax[1].legend(loc='best', shadow=True)

### Comments on third model

---

### Optimization of the third model

If we observe overfitting in the model, we can add `Dropout` as a regularization layer to prevent (or minimize) it. 

In this `model3d`, 25% of the nodes in the layer are randomly ignored in each epoch of training, allowing the network to learn different features.

In [ ]:
model3d = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (5,5), padding='same', activation='relu', input_shape=(28,28,1)),
    tf.keras.layers.Conv2D(64, (3,3), padding='same', activation='relu'),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Dense(num_classes, activation='softmax')
])

model3d.summary()

In [ ]:
model3d.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy']) # changed from rmsprop

### Callback

We can use the `ModelCheckpoint` callback function to keep track of model's parameters during training. The `save_best_only=True` option only saves when the model is considered the *best* and the latest best model according to the quantity monitored will not be overwritten. The best model is saved to a file and further loaded into a **new model** for performing prediction.

In [ ]:
from keras.callbacks import ModelCheckpoint

# training the model
checkpointer = ModelCheckpoint(filepath='mnist.model.best.hdf5', verbose=1, save_best_only=True)

We can also use a **larger batch size** for minimizing the training time.

In [ ]:
%time
history = model3d.fit(X_train2, y_train, batch_size=256, epochs=epochs, validation_split=0.3, callbacks=[checkpointer], shuffle=True)

In [ ]:
# evaluate test accuracy
score = model3d.evaluate(X_test2, y_test, verbose=0)
accuracy = 100*score[1]

In [ ]:
# print test accuracy
print('Test accuracy %.2f%%' % accuracy)

In [ ]:
fig, ax = plt.subplots(2,1)
ax[0].plot(history.history['loss'], color='b', label="Training Loss")
ax[0].plot(history.history['val_loss'], color='r', label="Validation Loss",axes =ax[0])
legend = ax[0].legend(loc='best', shadow=True)

ax[1].plot(history.history['accuracy'], color='b', label="Training Accuracy")
ax[1].plot(history.history['val_accuracy'], color='r',label="Validation Accuracy")
legend = ax[1].legend(loc='best', shadow=True)

### Loading the best model into a *new* model for prediction

In [ ]:
# load the weights that yielded the best validation accuracy
model3d.load_weights('mnist.model.best.hdf5')

In [ ]:
# evaluate test accuracy
score = model3d.evaluate(X_test2, y_test, verbose=0)
accuracy = 100*score[1]

# print test accuracy
print('Test accuracy %.2f%%' % accuracy)

### Students' activity (self-study)

Try to improve the model by adding more layers and/or increasing the batch size. You can also rerun all the code and play with other optimizer (e.g. `rmsprop` or `adam`).

Remember to use `X_train2` and `X_test2` as input data for training/validation and evaluation.

The model is able to achieve >98% accuracy depending on your choices.

We can also use a larger batch size for minimizing the training time.

In [ ]:
## model definition
model4 = ...

In [ ]:
## training
## you can decide on using shuffle and the ModelCheckpoint callback we've used before
%time
history = model4.fit(...)

In [ ]:
# evaluate test accuracy
score = model4.evaluate(...)
accuracy = 100*score[1]

In [ ]:
# print test accuracy
print('Test accuracy %.2f%%' % accuracy)

In [ ]:
fig, ax = plt.subplots(2,1)
ax[0].plot(history.history['loss'], color='b', label="Training Loss")
ax[0].plot(history.history['val_loss'], color='r', label="Validation Loss",axes =ax[0])
legend = ax[0].legend(loc='best', shadow=True)

ax[1].plot(history.history['accuracy'], color='b', label="Training Accuracy")
ax[1].plot(history.history['val_accuracy'], color='r',label="Validation Accuracy")
legend = ax[1].legend(loc='best', shadow=True)